<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 10: Bayesian Statistics and Likelihood</h1>


<a name='section_10_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.0 Overview</h2>


<table style="width:100%">
    <colgroup>
       <col span="1" style="width: 40%;">
       <col span="1" style="width: 15%;">
       <col span="1" style="width: 45%;">
    </colgroup>
    <tr>
        <th style="text-align: left; font-size: 13pt;">Section</th>
        <th style="text-align: left; font-size: 13pt;">Exercises</th>
        <th style="text-align: left; font-size: 13pt;">Summary</th>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_1">L10.1 Definition of Convolution</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_1">L10.1 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_2">L10.2 Example of Convolutions with Different Functions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_2">L10.2 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_3">L10.3 Prior and Posterior Probabilities and Bayes Theorem</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_3">L10.3 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_4">L10.4 Bayesian vs. Frequentist and Likelihood</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_4">L10.4 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_5">L10.5 Questions About Likelihood</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_5">L10.5 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_6">L10.6 Likelihood Ratio</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_6">L10.6 Exercises</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
</table>



<h3>Learning Objectives</h3>

Text needed


<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 
Optionally, set the plot resolution and default figure size.


In [7]:
#>>>RUN

import numpy as np

#set plot resolution
#%config InlineBackend.figure_format = 'retina'

#set default figure size
#plt.rcParams['figure.figsize'] = (9,6)


<a name='section_10_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.1 Definition of Convolution</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_0) | [Exercises](#exercises_10_1) | [Next Section](#section_10_2) |


<h3>Overview</h3>

Convlutions are a critical component of evey good statistical analysis. Its a way to multiply distributions together. 
Lets build convlutions up by scratch and then go from there. 

The core concept of a convolution is that you are effectively multiplying distributions. Given two functions $f(x)$ and $g(x)$, we can define convolutions by 
\begin{eqnarray}
(f*g)(z) &=& \int^{\infty}_{-\infty} f(z-t)g(t)dt
\end{eqnarray}
For data analysis, we usually think about this in the context of probability distribuitons $g$ and $f$. From here, we construct a new probability distribution $(f*g)$. Anyway, lets take a look at how it works. 

To do this, lets first define some functions to convolve. 

In [10]:
#>>>RUN

#First lets define a triangular distribution
def triangle(x,mean=5):
    Norm=mean*mean
    val=np.where(x <= mean,np.maximum(x,np.zeros(len(x))), np.maximum(2*mean-x,np.zeros(len(x))))
    return val/Norm

#Now define the gaussian
def gaussian(x,mean=0,sigma=1):
    return 1./(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * sigma**2)) 


In [10]:
#>>>RUN

#Now lets do a convolution by hand
def convolve(f1,f2,x,sigma=1,iMin=-10,iMax=10,iN=2000):
    step=(iMax-iMin)/iN
    pInt=0
    for i0 in range(iN):
            pX   = np.repeat(i0*step+iMin,len(x))
            pVal = f1(x-pX,sigma=sigma)*f2(pX)
            pInt += pVal*step
    return pInt

In [10]:
#>>>RUN

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
tri_out=triangle(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,triangle,x_in)
conv2_out=convolve(gaussian,triangle,x_in,sigma=2)

ax.plot(x_in,tri_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<a name='exercises_10_1'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_1) | [Next Section](#section_10_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.1.1</span>

text


In [13]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_10_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.2 Example of Convolutions with Different Functions</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_1) | [Exercises](#exercises_10_2) | [Next Section](#section_10_3) |


### Challenge question

What does $f(x)=\sin(x)$ convoled with a gaussian look like? How does it change by the resolution of the gaussian?

In [20]:
#>>>RUN

def func(x):
    return np.sin(x)*x

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
func_out=func(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,func,x_in)
conv2_out=convolve(gaussian,func,x_in,sigma=2)

ax.plot(x_in,func_out,label='sin(x)')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=2$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

In [20]:
#>>>RUN

#answer
def func(x):
    return np.sin(x)*x

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
sin_out=sin(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,sin,x_in,sigma=2)


ax.plot(x_in,sin_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<a name='exercises_10_2'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_2) | [Next Section](#section_10_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.2.1</span>

text


In [23]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_10_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.3 Prior and Posterior Probabilities and Bayes Theorem</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_2) | [Exercises](#exercises_10_3) | [Next Section](#section_10_4) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [1]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_03.html', width=970, height=550)

<h3>What is a measurment?</h3>

Lets say we have a measurement of some parameter $x$, and this meausurement behaves like a gaussian about some point with width $\sigma=1$. Lets write out this meausrement.  

In [20]:
#>>>RUN

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#Our measurement probability
def gaus(mu=0,sigma=1): 
    x = np.arange(-10, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    return x,y

#a quick plot of what we expect the measurement to be
def plotgaus():
    x,y=gaus(0,1)
    fig, ax = plt.subplots(figsize=(9,6))
    plt.style.use('fast')
    ax.plot(x,y)
    ax.fill_between(x,y,0, alpha=0.1, color='b')
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    plt.show()

plotgaus()

This preconceived distribution about how our measurement will behave is known as a prior. Now lets say we perform this measurement, but our observed measurement is not actually at the point we expect it to be. What if, for example our measurement is at $x=2$ what woud be the likelihood of this occuring or not occuring. 

In [20]:
#>>>RUN

def gaus(mu=0,sigma=1,meas=2): 
    x = np.arange(-10, 10, 0.001)
    xmeas = np.arange(meas, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    ymeas = stats.norm.pdf(xmeas,mu,sigma)
    return x,y,xmeas,ymeas

def plotgaus():
    x,y,xmeas,ymeas=gaus(0,1,2)
    fig, ax = plt.subplots(figsize=(9,6))
    plt.style.use('fast')
    ax.plot(x,y)
    ax.fill_between(x,y,0, alpha=0.1, color='b')
    ax.fill_between(xmeas,ymeas,0, alpha=0.3, color='b')
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    plt.show()
plotgaus()

From the above, observed measurement. It is clear that this distribution has a large deviation from what we actually expected to observe. The key question here that we would like to understand: Is this a statistical fluctation, or is this something we did not expect?  

Understanding if this is something we do not expect is the focus of this lecture. Lets now imagine that we perform this measurement a number of time, say 10 times, and it all comes up away from our expectation, or perhaps 1000 times, what do we thing?

In [20]:
#>>>RUN

def gaus(mu=0,sigma=1): 
    x = np.arange(-10, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    return x,y

def plotGausSample(iZ,iSample):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #sample
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #
    xs,ys=gaus(0,1)
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    count, bins, ignored = plt.hist(samples, 30, density=True)
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()

plotGausSample(2,10)
plotGausSample(2,1000)

Now we have two results, we have a prior (aka a guess) of what we though the data would look like and a posterior for what we actually observe the data. These two observations yield two different ways to interpret the data. 

**Bayesian** It is possible to create a model of everything and within your model you can explain all random phenomena. As we take more data, we can fine tune our model to be ever more predictive. 

**Frequentist** The data guides our model. We can use what we observe in the data to explain how we will observe these phenomena in the future. 

<h3>Bayes Thereom</h3>

To understand Bayes theorem, lets define a bunch of terms, first we would like the define the probability of a hypothesis happening. We can write this probability as $P\left(\mathcal{H}\right)$, this is the probability of the hypothesis happening given a prior for how we expect the distribution to behave. To put a concrete label to this, lets consider that you are on a game show similar to the "Monty Hall" game show of the 1960s. You have 3 doors and there is a car behind one of the doors. The car is placed randomly, what is your original $P\left(\mathcal{H}\right)$?

\begin{eqnarray}
P(\mathcal{H}=\rm{door~1}) & = & \frac{1}{3} \\
P(\mathcal{H}=\rm{door~2}) & = & \frac{1}{3} \\
P(\mathcal{H}=\rm{door~3}) & = & \frac{1}{3} 
\end{eqnarray}

Now everything changes when you open one of the doors. In that instance your knowledge of what happens changes completely since you now have knowledge of what is behind which door. In the case of this, there are two results. The first result is that there is a car behind the door, great! In the second case, you know that the door that was opened is empty. So the question is what do you.  However, in the Monty Hall problem. Monty will mix things up by opening a door that does not have a car, and then asking you to stay with you original choice, or to switch. What do you do? 

Let's define $P\left(\mathcal{D}\right)$ as the probability of an instance of the data happening, and furthemore defining

\begin{eqnarray}
 P\left(\mathcal{H} | \mathcal{D} \right) & = & \rm{probability~of~a~hypothesis~given~data} \\
 P\left(\mathcal{D} | \mathcal{H} \right) & = & \rm{probability~of~data~given~a~hypothesis} \\
\end{eqnarray}

Let's say you choose door 1, and Monty opens door two. We can write down the probability that Monty will open door 2
\begin{eqnarray}
P(\rm{open~2}|\mathcal{H}=\rm{car~at~door~1}) & = & \frac{1}{2} \\
P(\rm{open~2}|\mathcal{H}=\rm{car~at~door~2}) & = & 0 \\
P(\rm{open~2}|\mathcal{H}=\rm{car~at~door~3}) & = & 1 
\end{eqnarray}

If you chose correctly, the first time, then Monty has a 50% chance of choosing door 2 or 3, and both doors will not contain a car. If you did not choose correctly the first time, then Monty will for sure open a specific door, and in this scenario you know for sure that behind the other door that Monty did not choose is the car. So given this perspective do you switch or stay? Lets write out the probabilities again, remember when you start you have a $\frac{1}{3}$ chance of choosing correctly, and a $\frac{2}{3}$ chance of choosing incorrectly. 

\begin{eqnarray}
P(\mathcal{H}=\rm{car~at~door~1}|\mathcal{D}=\rm{open~2}) & = & \frac{1}{3} \\
P(\mathcal{H}=\rm{car~at~door~2}|\mathcal{D}=\rm{open~2}) & = & 0 \\
P(\mathcal{H}=\rm{car~at~door~3}|\mathcal{D}=\rm{open~2}) & = & \frac{2}{3}  
\end{eqnarray}

or more generically we can write out:
\begin{equation}
P(\mathcal{H}=\rm{car~at~door~1}|\mathcal{D}=\rm{open~2}) = \frac{P(\mathcal{D}=\rm{open~2}|\mathcal{H}=\rm{car~at~door~1})P(\mathcal{H}=\rm{door~1})}{\rm{all~combinations}}
\end{equation}
or in other words

\begin{equation}
P(\mathcal{H}=\rm{car~at~door~1}|\mathcal{D}=\rm{open~2}) = \frac{P(\mathcal{D}=\rm{open~2}|\mathcal{H}=\rm{car~at~door~1})P(\mathcal{H}=\rm{door~1})}{P\left(\mathcal{D}=\rm{car~at~door~1}\right)+P\left(\mathcal{D}=\rm{car~at~door~2}\right)+P\left(\mathcal{D}=\rm{car~at~door~3}\right)}
\end{equation}

More generically, we can write this as what is known as Bayes theorem
\begin{equation}
 P\left(\mathcal{H} | \mathcal{D} \right) =  \frac{P\left(\mathcal{D} | \mathcal{H} \right)P(\mathcal{H})}{P(\mathcal{D})} 
\end{equation}

We can also relabel these terms as: 
\begin{eqnarray}
 P\left(\mathcal{H} | \mathcal{D} \right) & = & \rm{Posterior} \\
 P\left(\mathcal{D} | \mathcal{H} \right) & = & \rm{Likelihood} \\
 P\left(\mathcal{H} \right) & = & \rm{Prior} \\
 P\left(\mathcal{D} \right) & = & \rm{Normalizer(all~possibilities)} 
\end{eqnarray} 

The Posterior is our observed result, the Prior is our initial guess, the likelihood is what actually we observe, and finally the bottom term, the normalizer, is to ensure that our probabilities integrate to $1$ (aka, we have covered, and only covered, all possibilities).  As a general rule of thumb, the way to remember this is

\begin{equation}
 \rm{posterior} \propto \rm{likelihood} \times \rm{prior}
\end{equation}

Given Bayes theorem, lets go back to our original measurement. In that case, we had two normal distributions, a posterior about two, and a prior about zero. How do we connect these two? 

\begin{equation}
 P\left(\mathcal{H} | \mathcal{D} \right) =  \frac{P\left(\mathcal{D} | \mathcal{H} \right)P(\mathcal{H})}{P(\mathcal{D})} \\
\mathcal{N}(x,\mu=2,1) = \frac{P\left(\mathcal{D} | \mathcal{H} \right)}{P(\mathcal{D})}\mathcal{N}(x,\mu=0,1) \\
P\left(\mathcal{D} | \mathcal{H} \right) = \frac{\mathcal{N}(x,\mu=2,1)}{\mathcal{N}(x,\mu=0,1)} P(\mathcal{D}) 
\end{equation}

Since $P(\mathcal{D})$ is just a constant to ensure that our resulting probability is normalized, we can pretty easily compute the likelihood in our first example. Lets plot it. 

In [20]:
#>>>RUN

def plotGausSampleLike(iZ,iSample):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #Sample
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #prior
    xs,ys=gaus(0,1)
    #likelihood
    yratio=np.minimum(y/ys,1.)
    #plot
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    ax.plot(x,yratio,label='Likelihood/20')
    count, bins, ignored = plt.hist(samples, 30, density=True)
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()

plotGausSampleLike(2,100)

As you can see the likelihood shoots up, and in fact shoots up above 1, this means that our liklelihood is unphysical, and we are not capturing our physics, or more imporantly, this means that **our prior is wrong**. Whats the right prior in this scenario? 

Lets tweak our prior to have a large sigma $\mathcal{N}(\mu=0,\sigma=2)$

In [20]:
#>>>RUN

def plotGausSampleLike(iZ,iSample,iSigMax):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #ample our posterior
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #Sample our prior
    xs,ys=gaus(0,iSigMax) #######<<<<< This is our tweak
    #now compute the likelihood
    yratio=np.minimum(y/ys,20.)
    #pot this stuff
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    ax.plot(x,yratio,label='Likelihood/20')
    count, bins, ignored = plt.hist(samples, 30, density=True)
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()
    #now return our sampled normal distribution
    return samples
    
samples=plotGausSampleLike(2,100,2)

What have we done?  Essentially, we had previously claimed that our measurement was at zero with uncertainty (expected $\sigma$=1). Now what we have done is made the claim that our uncertainty is way larger. What this has done is lowered our likelihood for something observed to happen to some number that is now not insanely large. 

### Challenge question

Consider that we have two gaussians one with mean of 0 and one with mean of 0.1, and both have width 1. In one experiment, we sample it 1000 samples, and in another we sample it 1,000,000 times. What is the max likelihood value of all the observed points? What would the case be if we sampled even more (10M...)? What does this mean about the distributions? 

In [20]:
#>>>RUN

#solution
def maxlikelihood(isamples,mu1=0.2,sig1=1,mu2=0,sig2=1):
    val=np.max(isamples)#compute the highest sampled gaussian
    #now compute the liklihood of these two
    like=stats.norm.pdf(val,mu1,sig1)/stats.norm.pdf(val,mu2,sig2)
    return like

def maxlike(iN):
    samples=plotGausSampleLike(0.2,iN,1)
    like=maxlikelihood(samples)
    print("Max:",iN," is ",like)

maxlike(1000)
maxlike(1000000)

<a name='exercises_10_3'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_3) | [Next Section](#section_10_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.3.1</span>

text


In [33]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_10_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.4 Bayesian vs. Frequentist and Likelihood</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_3) | [Exercises](#exercises_10_4) | [Next Section](#section_10_5) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [2]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_04.html', width=970, height=550)

<h3>Overview</h3>

Now, this interpretation all comes to a head when we start to think about how we wish to quote our results. If our priors are incorrect, and we continue to progressively take data, at some point our priors are going to be wrong. Dealing with how this is wrong depends on what sort of statistician you are.  See the diagram in the first slide above.

**Frequentist** If I see a data distribution, I can come up with a way to fit it. By getting a good fit, I can explain the next result. 

**Bayesian** I start with a model, by taking more and more measurements, I can improve my model. 

To try to capture these ideas, lets fit this guassian with two separate approaches. 

In [20]:
#>>>RUN

#with Bayesian, we hypothesize a guassian and fit it
from lmfit.models import GaussianModel
#randomly sample 100 events froma. gaussian
lN=100
samples = np.random.normal(0,1,lN)
#make a histogram
count, bins, ignored = plt.hist(samples,30)
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])
#poisson unc.
weight=1./np.sqrt(count)
weight[weight==float('+inf')] = 0
plt.show()

#Now we can consider two ways to interpret the data
def frequentist(iBins,iCount,weight): #fit a gaussian float all parameters
    model = GaussianModel()
    params = model.make_params(center=2, amplitude=1, sigma=1) 
    result = model.fit(iCount, params, x=iBins,weights=weight)
    result.plot()
    print(result.fit_report())
    
def bayesianBad(iBins,iCount,weight):#fit a gaussian fix the mean and sgima
    model = GaussianModel()
    params = model.make_params(center=2, amplitude=1, sigma=1) 
    params['center'].vary=False
    params['sigma'].vary=False
    result = model.fit(iCount, params, x=iBins,weights=weight)
    result.plot()
    print(result.fit_report())

frequentist(binscenters,count,weight)
bayesianBad(binscenters,count,weight)

From above, what you see is that if we have a sample data, in the frequentist scenario, we just fit this distribution, and extract the parameters. In the frequentist approach, the data is key, and so if our $\chi^{2}$ is good for our fitted model, we can declare success. 

In the Bayesian approach, we need to reocncile our prior with our fitted data. If our prior is that our data should behave as a gaussian about two, and we try to fit it to the data, you see there is not a very good $\chi^{2}$ value, and the fit is clearly off. 

To reconcile our Bayesian fit, what we need to do is modify our model so that we can actually go from our prior to our fitted function. To do this, we need to insert a new prior. In this case, our prior will be that the mean of the Gaussian can vary. Let's write this down. 

\begin{eqnarray}
 P\left(\mathcal{H}=x\right|\mu,\sigma) & = & \mathcal{N}(x,\mu=2,\sigma=1) \\
 P\left(\mathcal{H}=\mu\right|\sigma) & = & \frac{1}{b-a}~\forall~\mu~\in~[a,b] \\ 
                               & = & 0~~~~~~~~~\forall~\mu~\notin~[a,b]
\end{eqnarray}
All that we are saying above is that mu can now vary between a and b. Lets now fit the data with this statement. To do this, we are going to use a new feature in `lmfit`. What we are aroing to do is call `lmfit.minimize` and feed it a modified loss, which we define as `resid` (aka the residual function).

Additionally, we will add another constraint and put it into the fit. In particular, we will add a paramater $\Delta_{\mu}$, such that the loss and $\mu$ will be written as: 
\begin{eqnarray}
\mu_{\rm new} & = & \mu -\Delta_{\mu} \\
\mathcal{L}_{\rm new} & = & \mathcal{L} + 0~\forall~\mu~\in~[a,b] \\ 
                      & = & \mathcal{L} + \inf~\forall~\mu~\notin~[a,b] 
\end{eqnarray}
In this case, we will just approximate a really large number as infinity. 

In [20]:
#>>>RUN

import lmfit
#here is our modified function
def gauss(x, amp, mu, sigma,dmu):
    return amp * np.exp(-(x-mu+dmu)**2 / (2.*sigma**2))

#now we define our loss we want to minimize
def resid(params, x, ydata,weights):
    mu    = params['center'].value
    sigma = params['sigma'].value
    amp   = params['amplitude'].value
    dmu   = params['deltamu'].value
    lossshift=0
    if abs(dmu) > 3:
        lossshift=1e32
    y_model= gauss(x,amp,mu,sigma,dmu)
    residarr = (y_model - ydata)*weights
    #now append our constraint to the loss
    residarr = np.append(residarr,lossshift)
    return residarr
    
def bayesianGood(iBins,iCount,weights):
    model = GaussianModel()
    params = model.make_params(center=2, amplitude=1, sigma=1) 
    params['center'].vary=False
    params['sigma'].vary=False
    params.add("deltamu", value=0.0, min=-10, max=10) #Our new line of code
    result = lmfit.minimize(resid, params, args=(iBins, iCount,weights))
    lmfit.report_fit(result)
    #Now we plot it. 
    plt.errorbar(iBins, iCount,np.sqrt(iCount), lw=2,fmt=".k", capsize=0)
    plt.plot(binscenters,gauss(binscenters,result.params['amplitude'].value,result.params['center'].value,result.params['sigma'].value,result.params['deltamu'].value))
    plt.xlabel("x")
    plt.ylabel("p")
    plt.show()

bayesianGood(binscenters,count,weight)



<a name='exercises_10_4'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_4) | [Next Section](#section_10_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.4.1</span>

text


In [43]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_10_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.5 Questions About Likelihood</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_4) | [Exercises](#exercises_10_5) | [Next Section](#section_10_6) |


<h3>Overview</h3>

TEXT


In [50]:
#>>>RUN

#code


<a name='exercises_10_5'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_5) | [Next Section](#section_10_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.5.1</span>

text


In [53]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_10_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.6 Likelihood Ratio</h2>     

| [Top](#section_10_0) | [Previous Section](#section_10_5) | [Exercises](#exercises_10_6) |


<h3>Slides</h3>

Run the code below to view the slides for this section.

In [3]:
#>>>RUN

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_06.html', width=970, height=550)

<a name='exercises_10_6'></a>   

| [Top](#section_10_0) | [Restart Section](#section_10_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 10.6.1</span>

text


In [63]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>
